In [ ]:
!pip install selenium
!pip install beautifulsoup4
!pip install pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

In [ ]:
url="https://pib.gov.in/indexd.aspx"

response=requests.get(url)
response

In [ ]:
soup=BeautifulSoup(response.text,'html')
soup

In [ ]:
soup.prettify

In [ ]:
press=soup.find_all('li',class_='has-sub')[1]
print(press)

In [ ]:
a=press.find_all("li")[1]

In [ ]:
a

In [ ]:
for i in a.find_all('a', href=True):
    print(i['href'])

In [ ]:
url1="https://pib.gov.in"

In [ ]:
response1 = requests.get(url1 + i["href"].replace("pib.gov.in", ""))
response1

In [ ]:
soup1=BeautifulSoup(response1.text,'html')

In [ ]:
soup1.prettify()

In [ ]:
soup1.get_text

In [ ]:
table = soup1.select('select',{'id': 'ContentPlaceHolder1_ddlMinistry'})

In [ ]:
ministry=[option.get_text(strip=True) for option in table[2].find_all('option')][1:]

In [ ]:
date=[option.get_text(strip=True) for option in table[3].find_all('option')][1:]

In [ ]:
month=[option for option in table[4].find_all('option')]
month=[i.text for i in month][3:]

In [ ]:
year=[option.get_text(strip=True) for option in table[5].find_all('option')][1]

In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame(columns=["Data/Time","Header","Content"])

In [ ]:
df

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup

def get_data(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1200x600')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    driver.implicitly_wait(10)
    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'html.parser')

    title_element = driver.find_element(By.XPATH, '//div[@class="text-center"]')
    date_element = driver.find_element(By.XPATH, '//div[@class="ReleaseDateSubHeaddateTime text-center pt20"]')
    content_elements = driver.find_elements(By.XPATH, '//p/span')

    title = title_element.text.strip()
    date = date_element.text.strip()
    content = ""
    for element in content_elements:
        if element.text.strip()=="*":
            break
        else:
            content+= element.text.strip()+"\n"

    # Print the result
    result = [date,title,content]
    return result

    # Close the browser
    driver.quit()

def update_date():
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in headless mode
    chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration (necessary in headless mode)
    chrome_options.add_argument('--window-size=1200x600')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://pib.gov.in/allRel.aspx")

    # Select values in the dropdowns
    # Replace 'option_value' with the value of the option you want to select

    select_element = Select(driver.find_element("id","ContentPlaceHolder1_ddlday"))  # Replace 'dropdown_id' with the actual ID of the dropdown element
    select_element.select_by_value("1")  # Replace 'option_value' with the value of the option you want to select

    select_element = Select(driver.find_element("id","ContentPlaceHolder1_ddlMonth"))
    select_element.select_by_visible_text("April")

    select_element = Select(driver.find_element("id","ContentPlaceHolder1_ddlYear"))
    select_element.select_by_value("2023")


    # Sleep to allow the website to update automatically (adjust this based on your website behavior)
    time.sleep(3)

    # Get and print the modified HTML after changes
    modified_html = driver.page_source

    # Close the webdriver
    driver.quit()

    return modified_html

response2=update_date()
soup2= BeautifulSoup(response2, 'html.parser')
row= soup2.find_all("div",class_="content-area")
href_links = []
for links1 in row:
    test=links1.select(".num li")
    for j in test:
        links =j.find_all('a', href=True)
        href_links.extend([link['href'] for link in links])
    for links2 in href_links:
        updated_url=url1+links2
        response3=requests.get(updated_url)
        soup_=BeautifulSoup(response3.text,'html')
        result1=get_data(updated_url)
        df.loc[len(df)]=result1
    df.drop_duplicates().reset_index(drop=True)

In [ ]:
df

In [ ]:
excel_file_path ='pib_test.xlsx'
df.to_excel(excel_file_path, index=False)